<a href="https://colab.research.google.com/github/Feuriee/Simulasi-Vigenere-Cipher-dengan-Teknik-Kasiski/blob/main/Algoritma_Kasiski.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kriptanalisis Kasiski untuk Vigenère Cipher**

**Dibuat dalam Google Colab**

## Import Model Pyhton

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import re
from math import gcd
from functools import reduce

## Menentukan **Frekuensi Alfabet Indonesia**

**Data frekuensi huruf dalam bahasa Indonesia**



In [ ]:
frekuensi_alfabetindonesia = {
    'A': 20.39, 'B': 2.64, 'C': 0.76, 'D': 5.00, 'E': 8.28,
    'F': 0.21, 'G': 3.66, 'H': 2.74, 'I': 7.98, 'J': 0.87,
    'K': 5.14, 'L': 3.26, 'M': 4.21, 'N': 9.33, 'O': 1.26,
    'P': 2.61, 'Q': 0.01, 'R': 4.64, 'S': 4.15, 'T': 5.58,
    'U': 4.62, 'V': 0.18, 'W': 0.48, 'X': 0.03, 'Y': 1.88,
    'Z': 0.04
}

#### Memisahkan keys dan values


In [ ]:
huruf = list(frekuensi_alfabetindonesia.keys())
frekuensi = list(frekuensi_alfabetindonesia.values())

#### Membuat histogram


In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(huruf, frekuensi, color='skyblue', edgecolor='black')
plt.title('Frekuensi Huruf dalam Bahasa Indonesia')
plt.xlabel('Huruf')
plt.ylabel('Frekuensi (%)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## Memasukan Input Chiper Text

In [ ]:
cipher_text = input("Masukan Chiper Text: ").lower()
print("\nChiper Text yang dimasukkan adalah:\n", cipher_text)

## Membersihkan teks (menghilangkan tanda baca)

In [334]:
clean_text = re.sub(r'[^a-z]', '', cipher_text)

## Analisis huruf yang muncul

In [340]:
frekuensi_huruf = Counter(clean_text)
sorted_huruf = sorted(frekuensi_huruf.items())
total_huruf, total_frekuensi = zip(*sorted_huruf)
frekuensi = {}
for char in clean_text:
    char_upper = char.upper()
    if char_upper in frekuensi:
        frekuensi[char_upper] += 1
    else:
        frekuensi[char_upper] = 1

## Tampilkan jumlah tiap huruf

In [ ]:
print("\nFrekuensi tiap huruf (A-Z):")
for huruf in sorted(frekuensi):
    print(f"{huruf} = {frekuensi[huruf]}")

## Tampilkan histogram frekuensi huruf

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(total_huruf, total_frekuensi, color='salmon')
plt.title('Histogram Frekuensi Huruf')
plt.xlabel('Huruf')
plt.ylabel('Frekuensi')
plt.tight_layout()
plt.show()

## Menentukan Semua Trigram dan posisi

In [343]:
trigram_positions = defaultdict(list)

In [344]:
for i in range(len(clean_text) - 2):
    trigram = clean_text[i:i+3]
    trigram_positions[trigram].append(i)

## Filter trigram yang muncul lebih dari sekali

In [345]:
repeated_trigrams = {k: v for k, v in trigram_positions.items() if len(v) > 1}

## Tampilkan hasil

In [ ]:
if repeated_trigrams:
    print("\n=== Hasil Analisis Kasiski (Trigram) ===")
    for trigram, positions in repeated_trigrams.items():
        print(f"\nTrigram: {trigram}")
        distances = []
        for i in range(len(positions) - 1):
            distance = positions[i + 1] - positions[i]
            distances.append(distance)
            print(f"  Posisi: {positions[i]} -> {positions[i+1]} | Jarak: {distance}")
        if len(distances) > 1:
            key_length_est = reduce(gcd, distances)
            print(f"  Estimasi panjang kunci (GCD jarak): {key_length_est}")
        elif distances:
            print(f"  Jarak satu-satunya: {distances[0]}")
else:
    print("\nTidak ditemukan trigram yang berulang dalam teks.")

## menentukan panjang kunci melalui jarak pada trigram

In [ ]:
all_distances = []
for trigram, positions in repeated_trigrams.items():
    for i in range(len(positions) - 1):
        distance = positions[i + 1] - positions[i]
        all_distances.append(distance)

if all_distances:
    print("\n=== Semua Jarak yang Ditemukan ===")
    print("Jarak:", all_distances)


##  Menentukan Faktor Persekutuan Terbesar (FPB) dari seluruh jarak tersebut

In [ ]:
def faktor(n):
    return [i for i in range(1, n + 1) if n % i == 0]

print("\n=== Faktor dari Tiap Jarak ===")
for jarak in all_distances:
    faktors = faktor(jarak)
    faktors_str = ', '.join(str(f) for f in faktors)
    print(f"{jarak} = {faktors_str}")


## Menentukan nilai paling sering muncul dari Faktor Persekutuan Terbesar (FPB) jarak tersebut

In [ ]:
from collections import Counter

# Langkah 1: Kumpulkan semua faktor dari semua jarak
semua_faktor = []

for jarak in all_distances:
    semua_faktor.extend(faktor(jarak))  # tambahkan semua faktor dari setiap jarak

# Langkah 2: Hitung frekuensi tiap faktor
faktor_counter = Counter(semua_faktor)

# Langkah 3: Urutkan berdasarkan frekuensi tertinggi
sorted_faktor = sorted(faktor_counter.items(), key=lambda x: x[1], reverse=True)

# Langkah 4: Tampilkan hasil
print("\n=== Frekuensi Kemunculan Faktor ===")
for faktor_, freq in sorted_faktor:
    print(f"Faktor {faktor_} muncul sebanyak {freq} kali")

# Langkah 5: Ambil kandidat panjang kunci (bisa 2–3 teratas)
print("\n=== Kandidat Panjang Kunci (Faktor Terbanyak) ===")
for faktor_, freq in sorted_faktor:
    if freq >= 2:  # ambil yang muncul lebih dari 1 kali
        print(f"➡️ {faktor_} (muncul {freq} kali)")


## Masukkan panjang kunci yang sudah kamu dapatkan

In [350]:
while True:
    try:
        panjang_kunci_input = int(input("Masukkan estimasi panjang kunci (bilangan bulat positif): "))
        if panjang_kunci_input > 0:
            panjang_kunci_estimasi = panjang_kunci_input
            break
        else:
            print("Panjang kunci harus bilangan bulat positif.")
    except ValueError:
        print("Input tidak valid. Harap masukkan bilangan bulat.")

print(f"\nMenggunakan estimasi panjang kunci: {panjang_kunci_estimasi}")

Masukkan estimasi panjang kunci (bilangan bulat positif): 7

Menggunakan estimasi panjang kunci: 7


## Bagi ciphertext menjadi beberapa grup, tiap grup berisi huruf-huruf yang dienkripsi dengan huruf kunci yang sama.

In [ ]:
def group_by_panjang_kunci_estimasi(clean_text, panjang_kunci_estimasi):
    groups = ['' for _ in range(panjang_kunci_estimasi)]

    for i, char in enumerate(clean_text):
        group_index = i % panjang_kunci_estimasi
        groups[group_index] += char

    return groups

# Menggunakan data yang sudah ada
groups = group_by_panjang_kunci_estimasi(clean_text, panjang_kunci_estimasi)

print("\nGrup berdasarkan panjang kunci (index 0 sampai", panjang_kunci_estimasi-1, "):")
for i, group in enumerate(groups, start=1):
    print(f"Grup {i}: {group}")


## Menentukan posisi tiap huruf pada grup dengan alfabet

**A B C D E F G H I J K L M N O P Q R S T U V W X Y Z**  
**1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26**


In [ ]:
def konversi_huruf(text):
    return [ord(c) - ord('a') + 1 for c in text]

# Misal groups sudah ada dari sebelumnya
for i, group in enumerate(groups, start=1):
    indices = konversi_huruf(group)
    # Format output: print grup + list indeks
    print(f"Grup {i}: {group} ({', '.join(map(str, indices))})\n")


## Mengambil frekuensi alfabet Bahasa Indonesia

In [353]:
freq_indo = [frekuensi_alfabetindonesia.get(chr(ord('A')+i), 0) for i in range(26)]


## Melakukan pergeseran (shift) tiap grup ciphertext berdasarkan panjang kunci

In [354]:
def shift_text(text, shift):
    # Geser huruf ke kiri sebanyak 'shift' posisi (mod 26)
    result = ""
    for c in text:
        # Convert huruf ke 0-25
        num = ord(c) - ord('a') + 1
        # geser
        num = (num - shift) % 26
        result += chr(num + ord('a'))
    return result

## Hitung nilai Chi-Square antara frekuensi huruf hasil shift dengan frekuensi huruf standar bahasa Indonesia.

**Rumus Chi-Square (χ²)**


 $\chi^2 = \sum_{i=1}^n \frac{(O_i - E_i)^2}{E_i}$


Keterangan:  
- \( $\chi^2 \$) = nilai Chi-Square  
- \( O_i \) = frekuensi observasi dari kategori ke-i  
- \( E_i \) = frekuensi ekspektasi dari kategori ke-i  
- \( n \) = jumlah kategori

In [355]:
def chi_square_statistik(observasi_freq, expected_freq, length):
    # Hitung Chi-Square antara distribusi observed dan expected
    chi_sq = 0
    for o, e in zip(observasi_freq, expected_freq):
        expected_count = e * length / 100  # e dalam persen → jumlah harapan
        if expected_count > 0:
            chi_sq += ((o - expected_count) ** 2) / expected_count
    return chi_sq


## Hitung frekuensi kemunculan huruf pada hasil shift.

In [356]:
def get_frequensi(text):
    length = len(text)
    count = Counter(text)
    # observed frekuensi jumlah huruf a-z
    observed = [count.get(chr(ord('a') + i), 0) for i in range(26)]
    return observed, length

## Hitung nilai Chi-Square antara frekuensi huruf hasil shift dengan frekuensi huruf standar bahasa Indonesia.

In [357]:
# Analisis Chi-Square tiap grup untuk cari shift terbaik
def find_best_shift_for_group(group_text):
    observasi, length = get_frequensi(group_text)
    min_chi = None
    best_shift = None

    for shift in range(26):
        # Geser frekuensi observed ke kanan (artinya dekode dengan shift)
        shifted_text = shift_text(group_text, shift)
        shifted_obs, _ = get_frequensi(shifted_text)

        chi = chi_square_statistik(shifted_obs, freq_indo, length)

        if (min_chi is None) or (chi < min_chi):
            min_chi = chi
            best_shift = shift

    return best_shift, min_chi

## menjalankan analisis untuk semua grup

In [ ]:
print("Analisis Chi-Square untuk tiap grup dan shift terbaik:")
kunci = ""

for i, group in enumerate(groups, start=1):
    shift, chi_val = find_best_shift_for_group(group)
    # Ubah shift 1-26 jadi huruf kunci, a=1
    # Jadi kita kurangi 1 saat konversi ke huruf: 1 -> 0 (a), 2 -> 1 (b), dst
    key_char = chr(((shift - 1) % 26) + ord('a'))
    print(f"Grup {i}: shift terbaik = {shift} (chi-square = {chi_val:.2f}), tebakan key = '{key_char}'")
    kunci += key_char

print("\nTebakan kunci lengkap:", kunci)